# 🌊 해상 탄성파 탐사 시뮬레이션
# Marine Seismic Survey Simulation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/knocgp/seismic/blob/main/Marine_Seismic_Standalone.ipynb)

---

## 📋 개요

**완전 독립 실행형 노트북** - 외부 파일 불필요!

이 노트북은 해상 탄성파 탐사의 전체 과정을 시뮬레이션합니다:

- ✅ 합성 지반 모델 생성 (해수층 포함)
- ✅ 반사법 기반 합성 탄성파 데이터 생성
- ✅ 해면 멀티플 시뮬레이션
- ✅ 내부 멀티플 시뮬레이션
- ✅ 5가지 해상 노이즈 추가
- ✅ 시각화 및 분석

### 🚀 사용법

1. **"런타임" > "모두 실행"** 클릭
2. 결과 확인 및 다운로드
3. 파라미터 변경하여 재실행

## 📦 1. 패키지 설치 및 임포트

In [ ]:
# 필요한 패키지 설치
!pip install -q numpy scipy matplotlib

# 라이브러리 임포트
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from typing import Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

print("✅ 패키지 설치 및 임포트 완료!")

## 🔧 2. 시뮬레이터 클래스 정의

In [ ]:
class MarineSeismicSimulator:
    """해상 탄성파 탐사 시뮬레이터"""
    
    def __init__(self, dt: float = 0.001, nt: int = 2000):
        """초기화"""
        self.dt = dt
        self.nt = nt
        self.time = np.arange(nt) * dt
        
    def create_synthetic_model(self, nlayers: int = 5) -> Dict:
        """합성 지반 모델 생성 (해수층 포함)"""
        model = {
            'velocity': [],
            'density': [],
            'thickness': [],
            'depth': [],
            'name': []
        }
        
        # 1. 해수층
        model['velocity'].append(1500.0)
        model['density'].append(1030.0)
        model['thickness'].append(500.0)
        model['depth'].append(0.0)
        model['name'].append('Water')
        
        # 2. 해저면 (연약 퇴적층)
        model['velocity'].append(1800.0)
        model['density'].append(2000.0)
        model['thickness'].append(300.0)
        model['depth'].append(500.0)
        model['name'].append('Soft sediment')
        
        # 3-N. 지하 지층들
        current_depth = 800.0
        for i in range(nlayers - 2):
            vp = 2000.0 + (i * 500.0) + np.random.normal(0, 100)
            rho = 2100.0 + (i * 100.0) + np.random.normal(0, 50)
            thickness = 200.0 + np.random.uniform(100, 300)
            
            model['velocity'].append(vp)
            model['density'].append(rho)
            model['thickness'].append(thickness)
            model['depth'].append(current_depth)
            model['name'].append(f'Layer {i+3}')
            
            current_depth += thickness
        
        return model
    
    def calculate_reflection_coefficients(self, model: Dict) -> Tuple[np.ndarray, np.ndarray]:
        """반사 계수 계산"""
        velocities = np.array(model['velocity'])
        densities = np.array(model['density'])
        thicknesses = np.array(model['thickness'])
        
        # 음향 임피던스
        impedance = velocities * densities
        
        # 반사 계수
        rc = np.zeros(len(velocities) - 1)
        for i in range(len(velocities) - 1):
            rc[i] = (impedance[i+1] - impedance[i]) / (impedance[i+1] + impedance[i])
        
        # 반사 시간 (양방향 주시)
        times = np.zeros(len(velocities) - 1)
        cumulative_time = 0
        for i in range(len(velocities) - 1):
            travel_time = thicknesses[i] / velocities[i]
            cumulative_time += travel_time
            times[i] = cumulative_time * 2
        
        return rc, times
    
    def ricker_wavelet(self, freq: float = 30.0) -> np.ndarray:
        """Ricker 파형 생성"""
        duration = 0.2
        t = np.arange(-duration/2, duration/2, self.dt)
        a = (np.pi * freq * t) ** 2
        wavelet = (1 - 2*a) * np.exp(-a)
        wavelet = wavelet / np.max(np.abs(wavelet))
        return wavelet
    
    def generate_synthetic_seismic(self, model: Dict, freq: float = 30.0) -> np.ndarray:
        """합성 탄성파 데이터 생성 (Reflectivity Method)"""
        rc, reflection_times = self.calculate_reflection_coefficients(model)
        
        # Reflectivity series
        reflectivity = np.zeros(self.nt)
        for rc_val, t_refl in zip(rc, reflection_times):
            idx = int(t_refl / self.dt)
            if idx < self.nt:
                reflectivity[idx] = rc_val
        
        # Wavelet 생성
        wavelet = self.ricker_wavelet(freq)
        
        # 컨볼루션
        seismic = signal.convolve(reflectivity, wavelet, mode='same')
        return seismic
    
    def add_sea_surface_multiple(self, seismic: np.ndarray, model: Dict, 
                                  strength: float = 0.5) -> np.ndarray:
        """해면 멀티플 추가"""
        water_depth = model['thickness'][0]
        water_velocity = model['velocity'][0]
        two_way_time = 2 * water_depth / water_velocity
        delay_samples = int(two_way_time / self.dt)
        sea_surface_rc = -0.95
        
        multiple = np.zeros_like(seismic)
        if delay_samples < len(seismic):
            multiple[delay_samples:] = seismic[:-delay_samples] * sea_surface_rc * strength
        
        if 2 * delay_samples < len(seismic):
            multiple[2*delay_samples:] += seismic[:-2*delay_samples] * (sea_surface_rc**2) * strength * 0.5
        
        return seismic + multiple
    
    def add_internal_multiples(self, seismic: np.ndarray, model: Dict,
                               strength: float = 0.3) -> np.ndarray:
        """내부 멀티플 추가"""
        result = seismic.copy()
        rc, reflection_times = self.calculate_reflection_coefficients(model)
        
        strong_reflectors = [(t, rc_val) for t, rc_val in zip(reflection_times, rc) 
                           if abs(rc_val) > 0.1]
        
        for i, (t1, rc1) in enumerate(strong_reflectors):
            for t2, rc2 in strong_reflectors[i+1:]:
                multiple_delay = t2 - t1 + (t2 - t1)
                delay_samples = int(multiple_delay / self.dt)
                
                if delay_samples < len(seismic):
                    multiple_strength = rc1 * rc2 * strength
                    result[delay_samples:] += seismic[:-delay_samples] * multiple_strength
        
        return result
    
    def add_marine_noise(self, seismic: np.ndarray, noise_level: float = 0.05) -> np.ndarray:
        """5가지 해상 노이즈 추가"""
        result = seismic.copy()
        signal_power = np.std(seismic)
        
        # 1. 백색 잡음
        white_noise = np.random.normal(0, noise_level * signal_power * 0.3, len(seismic))
        result += white_noise
        
        # 2. 선박 노이즈 (2-8 Hz)
        ship_freq = np.random.uniform(2, 8)
        ship_noise = noise_level * signal_power * 0.5 * np.sin(2 * np.pi * ship_freq * self.time)
        ship_noise *= (1 + 0.3 * np.sin(2 * np.pi * 0.5 * self.time))
        result += ship_noise
        
        # 3. 해양 환경 노이즈 (1-20 Hz)
        ocean_noise = np.zeros_like(seismic)
        for freq in np.random.uniform(1, 20, 5):
            amplitude = noise_level * signal_power * 0.2 * np.random.uniform(0.5, 1.0)
            ocean_noise += amplitude * np.sin(2 * np.pi * freq * self.time + 
                                             np.random.uniform(0, 2*np.pi))
        result += ocean_noise
        
        # 4. 스웰 노이즈 (0.1-0.5 Hz)
        swell_freq = np.random.uniform(0.1, 0.5)
        swell_noise = noise_level * signal_power * 0.4 * np.sin(2 * np.pi * swell_freq * self.time)
        swell_noise *= (1 + 0.5 * np.sin(2 * np.pi * 0.2 * self.time))
        result += swell_noise
        
        # 5. 버스트 노이즈
        num_bursts = np.random.randint(3, 8)
        for _ in range(num_bursts):
            burst_time = np.random.randint(0, len(seismic))
            burst_duration = np.random.randint(10, 50)
            if burst_time + burst_duration < len(seismic):
                burst = noise_level * signal_power * 2.0 * np.random.randn(burst_duration)
                result[burst_time:burst_time+burst_duration] += burst
        
        return result
    
    def plot_model(self, model: Dict):
        """지층 모델 시각화"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 8))
        
        depths = model['depth']
        velocities = model['velocity']
        densities = model['density']
        
        for i in range(len(depths)):
            depth_top = depths[i]
            depth_bottom = depths[i] + model['thickness'][i]
            
            ax1.fill_between([velocities[i]-100, velocities[i]+100], 
                            depth_top, depth_bottom, 
                            alpha=0.3, label=model['name'][i] if i < 5 else None)
            ax1.plot([velocities[i], velocities[i]], [depth_top, depth_bottom], 
                    'b-', linewidth=2)
            
            ax2.fill_between([densities[i]-50, densities[i]+50], 
                            depth_top, depth_bottom, 
                            alpha=0.3)
            ax2.plot([densities[i], densities[i]], [depth_top, depth_bottom], 
                    'r-', linewidth=2)
        
        ax1.set_xlabel('Velocity (m/s)', fontsize=12)
        ax1.set_ylabel('Depth (m)', fontsize=12)
        ax1.set_title('Velocity Model', fontsize=14, fontweight='bold')
        ax1.invert_yaxis()
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        ax2.set_xlabel('Density (kg/m³)', fontsize=12)
        ax2.set_ylabel('Depth (m)', fontsize=12)
        ax2.set_title('Density Model', fontsize=14, fontweight='bold')
        ax2.invert_yaxis()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def plot_seismic_comparison(self, seismic_data: Dict):
        """탄성파 데이터 비교"""
        n_traces = len(seismic_data)
        fig, axes = plt.subplots(1, n_traces, figsize=(5*n_traces, 10))
        
        if n_traces == 1:
            axes = [axes]
        
        for ax, (title, data) in zip(axes, seismic_data.items()):
            ax.plot(data, self.time, 'k-', linewidth=0.5)
            ax.fill_betweenx(self.time, 0, data, where=(data>0), 
                            color='black', alpha=0.5)
            
            ax.set_xlabel('Amplitude', fontsize=11)
            ax.set_ylabel('Time (s)', fontsize=11)
            ax.set_title(title, fontsize=13, fontweight='bold')
            ax.invert_yaxis()
            ax.grid(True, alpha=0.3)
            ax.set_xlim([-np.max(np.abs(data))*1.1, np.max(np.abs(data))*1.1])
        
        plt.tight_layout()
        plt.show()
    
    def plot_spectrum(self, seismic_data: Dict):
        """주파수 스펙트럼 분석"""
        fig, axes = plt.subplots(len(seismic_data), 1, figsize=(12, 4*len(seismic_data)))
        
        if len(seismic_data) == 1:
            axes = [axes]
        
        for ax, (title, data) in zip(axes, seismic_data.items()):
            freq = np.fft.rfftfreq(len(data), self.dt)
            spectrum = np.abs(np.fft.rfft(data))
            
            ax.plot(freq, spectrum, 'b-', linewidth=1.5)
            ax.set_xlabel('Frequency (Hz)', fontsize=11)
            ax.set_ylabel('Amplitude', fontsize=11)
            ax.set_title(f'Frequency Spectrum - {title}', fontsize=13, fontweight='bold')
            ax.grid(True, alpha=0.3)
            ax.set_xlim([0, 100])
        
        plt.tight_layout()
        plt.show()

print("✅ 시뮬레이터 클래스 정의 완료!")

## 🚀 3. 기본 시뮬레이션 실행

In [ ]:
print("="*70)
print("해상 탄성파 탐사 시뮬레이션 시작")
print("="*70)

# 시뮬레이터 초기화
simulator = MarineSeismicSimulator(dt=0.001, nt=2000)
print("\n✓ 시뮬레이터 초기화 완료")

# 1. 합성 지반 모델 생성
print("\n[1단계] 합성 지반 모델 생성...")
model = simulator.create_synthetic_model(nlayers=6)

print("\n📊 지층 정보:")
print(f"{'Layer':<15} {'Depth (m)':<12} {'Thickness (m)':<15} {'Velocity (m/s)':<15} {'Density (kg/m³)'}")
print("-"*75)
for i in range(len(model['name'])):
    print(f"{model['name'][i]:<15} {model['depth'][i]:<12.1f} {model['thickness'][i]:<15.1f} "
          f"{model['velocity'][i]:<15.1f} {model['density'][i]:<15.1f}")

# 모델 시각화
simulator.plot_model(model)

# 2. 합성 탄성파 데이터 생성
print("\n[2단계] 합성 탄성파 데이터 생성...")
clean_seismic = simulator.generate_synthetic_seismic(model, freq=30.0)
print("✓ Clean 탄성파 데이터 생성 완료")

# 3. 멀티플 추가
print("\n[3단계] 멀티플 추가...")
seismic_with_multiples = simulator.add_sea_surface_multiple(clean_seismic, model, strength=0.5)
seismic_with_multiples = simulator.add_internal_multiples(seismic_with_multiples, model, strength=0.3)
print("✓ 해면 멀티플 추가 완료")
print("✓ 내부 멀티플 추가 완료")

# 4. 노이즈 추가
print("\n[4단계] 해상 노이즈 추가...")
noisy_seismic = simulator.add_marine_noise(seismic_with_multiples, noise_level=0.08)
print("✓ 백색 잡음 추가")
print("✓ 선박 노이즈 추가")
print("✓ 해양 환경 노이즈 추가")
print("✓ 스웰 노이즈 추가")
print("✓ 버스트 노이즈 추가")

# 5. 결과 비교
print("\n[5단계] 결과 시각화...")
seismic_comparison = {
    'Clean Seismic': clean_seismic,
    'With Multiples': seismic_with_multiples,
    'With Noise': noisy_seismic
}

simulator.plot_seismic_comparison(seismic_comparison)

# 6. 주파수 스펙트럼 분석
print("\n[6단계] 주파수 스펙트럼 분석...")
simulator.plot_spectrum(seismic_comparison)

# 7. 데이터 저장
print("\n[7단계] 데이터 저장...")
np.savez('marine_seismic_data.npz',
         time=simulator.time,
         clean=clean_seismic,
         with_multiples=seismic_with_multiples,
         noisy=noisy_seismic,
         model=model)
print("✓ 데이터 저장 완료: marine_seismic_data.npz")

# 통계 정보
print("\n" + "="*70)
print("📈 데이터 통계")
print("="*70)
print(f"샘플링 간격: {simulator.dt*1000:.2f} ms")
print(f"총 시간: {simulator.time[-1]:.2f} s")
print(f"Clean 신호 RMS: {np.sqrt(np.mean(clean_seismic**2)):.6f}")
print(f"멀티플 포함 RMS: {np.sqrt(np.mean(seismic_with_multiples**2)):.6f}")
print(f"노이즈 포함 RMS: {np.sqrt(np.mean(noisy_seismic**2)):.6f}")
print(f"SNR: {20*np.log10(np.std(seismic_with_multiples)/np.std(noisy_seismic-seismic_with_multiples)):.2f} dB")

print("\n✅ 시뮬레이션 완료!")
print("="*70)

## ⚙️ 4. 커스텀 파라미터 시뮬레이션

아래 파라미터를 변경하여 다양한 시뮬레이션을 실행해보세요!

In [ ]:
# 🎯 파라미터 설정 (자유롭게 변경 가능!)
WAVELET_FREQ = 40.0         # Wavelet 주파수 (Hz)
MULTIPLE_STRENGTH = 0.7     # 멀티플 강도 (0~1)
NOISE_LEVEL = 0.12          # 노이즈 레벨 (0~1)
NUM_LAYERS = 7              # 지층 개수

print("🎯 커스텀 파라미터:")
print(f"  Wavelet 주파수: {WAVELET_FREQ} Hz")
print(f"  멀티플 강도: {MULTIPLE_STRENGTH}")
print(f"  노이즈 레벨: {NOISE_LEVEL}")
print(f"  지층 개수: {NUM_LAYERS}")
print()

# 시뮬레이션 실행
custom_sim = MarineSeismicSimulator(dt=0.001, nt=2000)
custom_model = custom_sim.create_synthetic_model(nlayers=NUM_LAYERS)
custom_clean = custom_sim.generate_synthetic_seismic(custom_model, freq=WAVELET_FREQ)
custom_mult = custom_sim.add_sea_surface_multiple(custom_clean, custom_model, strength=MULTIPLE_STRENGTH)
custom_mult = custom_sim.add_internal_multiples(custom_mult, custom_model, strength=MULTIPLE_STRENGTH*0.6)
custom_noisy = custom_sim.add_marine_noise(custom_mult, noise_level=NOISE_LEVEL)

# 결과 시각화
custom_sim.plot_seismic_comparison({
    'Clean': custom_clean,
    'With Multiples': custom_mult,
    'With Noise': custom_noisy
})

print("\n✅ 커스텀 시뮬레이션 완료!")

## 💾 5. 결과 다운로드

In [ ]:
from google.colab import files

# 생성된 데이터 파일 다운로드
try:
    files.download('marine_seismic_data.npz')
    print("✅ 데이터 파일 다운로드 시작!")
except:
    print("⚠️ 다운로드 실패: 먼저 시뮬레이션을 실행하세요.")

## 📊 6. 저장된 데이터 로드 예제

In [ ]:
# 저장된 데이터 로드
try:
    data = np.load('marine_seismic_data.npz', allow_pickle=True)
    
    time = data['time']
    clean = data['clean']
    noisy = data['noisy']
    model = data['model'].item()
    
    print("📂 데이터 로드 정보:")
    print(f"  시간 샘플: {len(time)}")
    print(f"  시간 범위: 0 ~ {time[-1]:.2f} s")
    print(f"  지층 수: {len(model['name'])}")
    
    # 간단한 플롯
    plt.figure(figsize=(12, 6))
    plt.plot(time, clean, label='Clean', alpha=0.7)
    plt.plot(time, noisy, label='Noisy', alpha=0.7)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.title('Seismic Traces Comparison')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("\n✅ 데이터 로드 및 시각화 완료!")
    
except FileNotFoundError:
    print("⚠️ 데이터 파일이 없습니다. 먼저 시뮬레이션을 실행하세요.")

## 📚 참고 자료

### 주요 개념

- **음향 임피던스**: Z = ρ × V (밀도 × 속도)
- **반사 계수**: RC = (Z₂ - Z₁) / (Z₂ + Z₁)
- **양방향 주시**: TWT = 2 × d / v
- **Ricker Wavelet**: (1 - 2a) × exp(-a), where a = (πft)²

### 해상 탄성파 특징

1. **해수층**: 속도 ~1500 m/s, 밀도 ~1030 kg/m³
2. **해면 반사**: RC ≈ -0.95 (강한 음의 반사)
3. **멀티플**: 해면 및 내부 경계면 다중 반사
4. **노이즈**:
   - 선박 노이즈: 2-8 Hz
   - 해양 환경: 1-20 Hz
   - 스웰: 0.1-0.5 Hz
   - 버스트: 간헐적 충격

### GitHub 저장소

🔗 https://github.com/knocgp/seismic

---

**Made with ❤️ for Marine Seismic Research**